In [1]:
%matplotlib notebook
import numpy as np
import tensorflow as tf
import pandas
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from speed_regression import training_data as td
from speed_regression import rnn
from scipy.ndimage.filters import gaussian_filter1d
nano_to_sec = 1e09

# configure the size of plot
fig_size = plt.rcParams['figure.figsize']
fig_size[0] = 12
fig_size[1] = 9

In [5]:
# load data
data_all = pandas.read_csv('../../data/motion_demo/normal/processed/data.csv')

feature = data_all[['gyro_x', 'gyro_y', 'gyro_z', 'linacce_x', 'linacce_y', 'linacce_z',
                   'grav_x', 'grav_y', 'grav_z']].values
ts = data_all['time'].values / nano_to_sec
position = data_all[['pos_x', 'pos_y', 'pos_z']].values
orientation = data_all[['ori_w', 'ori_x', 'ori_y', 'ori_z']].values
gravity = data_all[['grav_x', 'grav_y', 'grav_z']].values

target = td.compute_local_speed_with_gravity(ts, position, orientation, gravity)
target_smooth_sigma = 30.0
target = gaussian_filter1d(target, sigma=target_smooth_sigma, axis=0)

output_dim = 2
# load model
tfconfig = tf.ConfigProto(device_count = {'GPU': 0})
tf.reset_default_graph()
feature_rnn = feature.reshape([1, -1, 9])
target_rnn = target[:, [0, 2]].reshape([1, -1, output_dim])
model_path = '../../models/LSTM_0428_mid/model.tf-161400'
with tf.Session(config=tfconfig) as sess:
    loader = tf.train.import_meta_graph(model_path + '.meta')
    print('Loading')
    loader.restore(sess, model_path)
    x = tf.get_collection('rnn_input')[0]
    y = tf.get_collection('rnn_output')[0]
    regressed = tf.get_collection('regressed')[0]
    state_size = tf.get_collection('state_size')[0]
    # init_state = tf.get_collection('init_state')[0]
    total_loss = tf.get_collection('total_loss')[0]
    num_layer = tf.get_collection('num_layer')[0]
    target_mean =  np.array([tf.get_collection('target_mean_x')[0],
                             tf.get_collection('target_mean_z')[0]])
    target_variance = np.array([tf.get_collection('target_variance_x')[0], 
                                tf.get_collection('target_variance_z')[0]])
    
    state = tuple([(np.zeros([1, state_size]), np.zeros([1, state_size])) for i in range(num_layer)])
    print('regressing')
    predicted, loss = sess.run([regressed, total_loss], feed_dict={x: feature_rnn,
                                                                    y: target_rnn})
    sess.close()
predicted = np.array(predicted).reshape(-1, output_dim)
predicted = predicted * target_variance + target_mean
tf.reset_default_graph()
plt.figure('Prediction')
plt.subplot(211)
plt.plot(ts, target[:, 0])
plt.plot(ts, predicted[:, 0])
plt.legend(['Ground truth', 'predicted'])
plt.subplot(212)
plt.plot(ts, target[:, 2])
plt.plot(ts, predicted[:, 1])
plt.legend(['Ground truth', 'predicted'])

#print('Error (tensorflow):', loss)
print('Error x: (sklearn):', r2_score(predicted[50:-1:50, 0], target[50:-1:50, 0]))
print('Error z: (sklearn):', r2_score(predicted[50:-1:50, 1], target[50:-1:50, 2]))

Loading
[]


IndexError: list index out of range

In [8]:
# plot error curve
logs_small = np.genfromtxt('../../models/LSTM_0424_small/losses.txt')
logs_mid = np.genfromtxt('../../models/LSTM_0428_mid/losses.txt')
logs_full = np.genfromtxt('../../models/LSTM_0501_full/losses.txt')

print('Small:', logs_small[-1])
print('Medium:', logs_mid[-1])
print('Large:', logs_full[-1])
# x = np.arange(0, logs.shape[0])
# plt.figure('losses')
# for i in range(3):
#     plt.plot(x, logs[:, i], lw=2)
# plt.plot(x, np.average(logs, axis=1), lw=2)
# plt.legend(['Training', 'Validation_x', 'Validation_z', 'Validation_all'])

Small: [  0.1381857   17.73778897   0.74427214]
Medium: [  0.26016461  32.12010448   0.48871734]
Large: [  0.59057533  30.90661367   0.61855967]
